In [585]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

In [586]:
# The topic of interest.
topic_dir_precip = "daily/more_precip/historical/"
topic_dir_temp = "daily/kl/historical/"
topic_dir_merged_data = "merged_data/"
topic_dir_appended_data = "appended_data/"
topic_dir_station = "stations/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
# ftp_dir =  ftp_climate_data_dir + topic_dir
ftp_dir_precip =  ftp_climate_data_dir + topic_dir_precip
ftp_dir_temp =  ftp_climate_data_dir + topic_dir_temp
ftp_dir_temp_station = ftp_climate_data_dir + topic_dir_temp
ftp_dir_precip_station = ftp_climate_data_dir + topic_dir_precip

In [587]:
local_ftp_dir           = "data/original/DWD/"    # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir   = local_ftp_dir + topic_dir_station # Local directory where local station info is located
local_ftp_precip_ts_dir = local_ftp_dir + topic_dir_precip# Local directory where time series downloaded from ftp are located
local_ftp_temp_ts_dir   = local_ftp_dir + topic_dir_temp

local_generated_dir     = "data/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir       = local_generated_dir + topic_dir_station # Derived station data, i.e. the CSV file
local_ts_merged_dir     = local_generated_dir + topic_dir_merged_data # Parallel merged time series, wide data frame with one TS per column
local_ts_appended_dir   = local_generated_dir + topic_dir_appended_data # Serially appended time series, long data frame for QGIS TimeManager Plugin

In [588]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_precip_ts_dir)
print(local_ftp_temp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

data/original/DWD/
data/original/DWD/stations/
data/original/DWD/daily/more_precip/historical/
data/original/DWD/daily/kl/historical/

data/generated/DWD/
data/generated/DWD/stations/
data/generated/DWD/merged_data/
data/generated/DWD/appended_data/


In [589]:
import os
os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_precip_ts_dir,exist_ok = True)
os.makedirs(local_ftp_temp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

In [590]:
import ftplib
ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [591]:
ret = ftp.cwd(".")

In [592]:
def grabFile(ftpfullname,localfullname):
    try:
        ret = ftp.cwd(".") # A dummy action to chack the connection and to provoke an exception if necessary.
        localfile = open(localfullname, 'wb')
        ftp.retrbinary('RETR ' + ftpfullname, localfile.write, 1024)
        localfile.close()
    
    except ftplib.error_perm:
        print("FTP ERROR. Operation not permitted. File not found?")

    except ftplib.error_temp:
        print("FTP ERROR. Timeout.")

    except ConnectionAbortedError:
        print("FTP ERROR. Connection aborted.")

In [593]:
import pandas as pd
import os

def gen_df_from_ftp_dir_listing(ftp, ftpdir):
    lines = []
    flist = []
    try:    
        res = ftp.retrlines("LIST "+ftpdir, lines.append)
    except:
        print("Error: ftp.retrlines() failed. ftp timeout? Reconnect!")
        return
        
    if len(lines) == 0:
        print("Error: ftp dir is empty")
        return
    
    for line in lines:
#        print(line)
        [ftype, fsize, fname] = [line[0:1], int(line[31:42]), line[56:]]
#        itemlist = [line[0:1], int(line[31:42]), line[56:]]
#        flist.append(itemlist)
        
        fext = os.path.splitext(fname)[-1]
        
        if fext == ".zip":
            station_id = int(fname.split("_")[2])
            
        else:
            station_id = -1 
        
        flist.append([station_id, fname, fext, fsize, ftype])
        
        

    df_ftpdir = pd.DataFrame(flist,columns=["station_id", "name", "ext", "size", "type"])
    return(df_ftpdir)

In [594]:
df_ftpdir_temp = gen_df_from_ftp_dir_listing(ftp, ftp_dir_temp_station)
df_ftpdir_precip = gen_df_from_ftp_dir_listing(ftp, ftp_dir_precip_station)

In [595]:
df_ftpdir_temp.head(10)

,station_id,name,ext,size,type
0,-1,BESCHREIBUNG_obsgermany_climate_daily_kl_histo...,.pdf,74902,-
1,-1,DESCRIPTION_obsgermany_climate_daily_kl_histor...,.pdf,73419,-
2,-1,KL_Tageswerte_Beschreibung_Stationen.txt,.txt,267457,-
3,1,tageswerte_KL_00001_19370101_19860630_hist.zip,.zip,282024,-
4,3,tageswerte_KL_00003_18910101_20110331_hist.zip,.zip,887544,-
5,11,tageswerte_KL_00011_19800901_20181231_hist.zip,.zip,83682,-
6,44,tageswerte_KL_00044_19690101_20181231_hist.zip,.zip,416393,-
7,52,tageswerte_KL_00052_19690101_20011231_hist.zip,.zip,261747,-
8,61,tageswerte_KL_00061_19750701_19780831_hist.zip,.zip,32904,-
9,70,tageswerte_KL_00070_19730601_19860930_hist.zip,.zip,100354,-


In [596]:
df_ftpdir_precip.head(10)

,station_id,name,ext,size,type
0,-1,BESCHREIBUNG_obsgermany_climate_daily_more_pre...,.pdf,72261,-
1,-1,DESCRIPTION_obsgermany_climate_daily_more_prec...,.pdf,71026,-
2,-1,RR_Tageswerte_Beschreibung_Stationen.txt,.txt,1202111,-
3,1,tageswerte_RR_00001_19120101_19860630_hist.zip,.zip,109677,-
4,2,tageswerte_RR_00002_19510101_20061231_hist.zip,.zip,82951,-
5,3,tageswerte_RR_00003_18910101_20110331_hist.zip,.zip,162410,-
6,4,tageswerte_RR_00004_19510101_19791031_hist.zip,.zip,45468,-
7,6,tageswerte_RR_00006_19821101_20181231_hist.zip,.zip,38084,-
8,7,tageswerte_RR_00007_19510101_19960131_hist.zip,.zip,69540,-
9,8,tageswerte_RR_00008_19310101_19911231_hist.zip,.zip,88001,-


In [597]:
 #df_ftpdir["ext"]==".zip"
df_zips_temp = df_ftpdir_temp[df_ftpdir_temp["ext"]==".zip"]
df_zips_temp.set_index("station_id", inplace = True)
df_zips_temp.head(10)

,name,ext,size,type
station_id,,,,
1,tageswerte_KL_00001_19370101_19860630_hist.zip,.zip,282024,-
3,tageswerte_KL_00003_18910101_20110331_hist.zip,.zip,887544,-
11,tageswerte_KL_00011_19800901_20181231_hist.zip,.zip,83682,-
44,tageswerte_KL_00044_19690101_20181231_hist.zip,.zip,416393,-
52,tageswerte_KL_00052_19690101_20011231_hist.zip,.zip,261747,-
61,tageswerte_KL_00061_19750701_19780831_hist.zip,.zip,32904,-
70,tageswerte_KL_00070_19730601_19860930_hist.zip,.zip,100354,-
71,tageswerte_KL_00071_19861101_20181231_hist.zip,.zip,200328,-
72,tageswerte_KL_00072_19780901_19950531_hist.zip,.zip,155186,-


In [598]:
 #df_ftpdir["ext"]==".zip"
df_zips_precip = df_ftpdir_precip[df_ftpdir_precip["ext"]==".zip"]
df_zips_precip.set_index("station_id", inplace = True)
df_zips_precip.head(10)

,name,ext,size,type
station_id,,,,
1,tageswerte_RR_00001_19120101_19860630_hist.zip,.zip,109677,-
2,tageswerte_RR_00002_19510101_20061231_hist.zip,.zip,82951,-
3,tageswerte_RR_00003_18910101_20110331_hist.zip,.zip,162410,-
4,tageswerte_RR_00004_19510101_19791031_hist.zip,.zip,45468,-
6,tageswerte_RR_00006_19821101_20181231_hist.zip,.zip,38084,-
7,tageswerte_RR_00007_19510101_19960131_hist.zip,.zip,69540,-
8,tageswerte_RR_00008_19310101_19911231_hist.zip,.zip,88001,-
9,tageswerte_RR_00009_19920601_20101231_hist.zip,.zip,31560,-
10,tageswerte_RR_00010_19610101_20050831_hist.zip,.zip,66983,-


In [599]:
#Both station name of temperature and precipitation are extracted
station_fname_temp = df_ftpdir_temp[df_ftpdir_temp['name'].str.contains(station_desc_pattern)]["name"].values[0]
station_fname_precip = df_ftpdir_precip[df_ftpdir_precip['name'].str.contains(station_desc_pattern)]["name"].values[0]
print(station_fname_temp)
print(station_fname_precip)

KL_Tageswerte_Beschreibung_Stationen.txt
RR_Tageswerte_Beschreibung_Stationen.txt


In [600]:
print("grabFile: ")
print("From: " + ftp_dir_station + station_fname_temp)
print("To:   " + local_ftp_station_dir + station_fname_temp)
print("From: " + ftp_dir_station + station_fname_precip)
print("To:   " + local_ftp_station_dir + station_fname_precip)
grabFile(ftp_dir_temp_station + station_fname_temp, local_ftp_station_dir + station_fname_temp) 
grabFile(ftp_dir_precip_station + station_fname_precip, local_ftp_station_dir + station_fname_precip) 

grabFile: 
From: /climate_environment/CDC/observations_germany/climate/daily/kl/historical/KL_Tageswerte_Beschreibung_Stationen.txt
To:   data/original/DWD/stations/KL_Tageswerte_Beschreibung_Stationen.txt
From: /climate_environment/CDC/observations_germany/climate/daily/kl/historical/RR_Tageswerte_Beschreibung_Stationen.txt
To:   data/original/DWD/stations/RR_Tageswerte_Beschreibung_Stationen.txt


In [601]:
 # extract column names. They are in German (de)
# We have to use codecs because of difficulties with character encoding (German Umlaute)
import codecs

def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r","utf-8")
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en, parse_dates=["date_from","date_to"],index_col = 0)
    
    # write csv
    df.to_csv(csvfile, sep = ";")
    return(df)

In [602]:
basename_temp = os.path.splitext(station_fname_temp)[0]
df_stations_temp = station_desc_txt_to_csv(local_ftp_station_dir + station_fname_temp, local_station_dir + basename_temp + ".csv")
df_stations_temp.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
1,1937-01-01,1986-06-30,478,47.8413,8.8493,Aach,Baden-Württemberg
3,1891-01-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
11,1980-09-01,2020-02-19,680,47.9737,8.5205,Donaueschingen (Landeplatz),Baden-Württemberg
44,1969-01-01,2020-02-19,44,52.9336,8.2370,Großenkneten,Niedersachsen
52,1969-01-01,2001-12-31,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein


In [603]:
basename_precip = os.path.splitext(station_fname_precip)[0]
df_stations_precip = station_desc_txt_to_csv(local_ftp_station_dir + station_fname_precip, local_station_dir + basename_precip + ".csv")
df_stations_precip.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
1,1912-01-01,1986-06-30,478,47.8413,8.8493,Aach,Baden-Württemberg
2,1951-01-01,2006-12-31,138,50.8066,6.0996,Aachen (Kläranlage),Nordrhein-Westfalen
3,1891-01-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
4,1951-01-01,1979-10-31,243,50.7683,6.1207,Aachen-Brand,Nordrhein-Westfalen
6,1982-11-01,2020-02-19,455,48.8361,10.0598,Aalen-Unterrombach,Baden-Württemberg


In [604]:
#Index of the station present in Nordrhein are listed 
station_ids_selected_temp = df_stations_temp[df_stations_temp['state'].str.contains("Nordrhein")].index 
station_ids_selected_temp

Int64Index([    3,    98,   186,   326,   386,   390,   553,   554,   555,
              598,
            ...
            13696, 13700, 13713, 13901, 13952, 15000, 15120, 15190, 15200,
            15963],
           dtype='int64', name='station_id', length=112)

In [605]:
station_ids_selected_precip = df_stations_precip[df_stations_precip['state'].str.contains("Nordrhein")].index
station_ids_selected_precip

Int64Index([    2,     3,     4,    38,    42,    79,    98,   104,   110,
              186,
            ...
            15456, 15559, 15919, 15927, 15963, 15980, 16087, 19042, 19043,
            19125],
           dtype='int64', name='station_id', length=500)

In [606]:
# Create variable with TRUE if state is Nordrhein-Westfalen
isNRW = df_stations_temp['state'] == "Nordrhein-Westfalen"

# Create variable with TRUE if date_to is latest date (indicates operation up to now)
isOperational = df_stations_temp['date_to'] == df_stations_temp.date_to.max() 

isBefore2010 = df_stations_temp['date_from'] < '2010'

# select on both conditions
dfNRW_temp = df_stations_temp[isNRW & isOperational & isBefore2010]
#print("Number of stations in NRW: \n", dfNRW.count())
dfNRW_temp

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
390,1986-12-01,2020-02-19,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
617,2004-06-01,2020-02-19,47,51.8730,6.8863,Borken in Westfalen,Nordrhein-Westfalen
1078,1952-01-01,2020-02-19,37,51.2960,6.7686,Düsseldorf,Nordrhein-Westfalen
1300,1938-01-01,2020-02-19,351,51.2540,8.1565,Eslohe,Nordrhein-Westfalen
1303,1935-01-01,2020-02-19,150,51.4041,6.9677,Essen-Bredeney,Nordrhein-Westfalen
1327,1937-01-01,2020-02-19,147,50.7119,6.7905,Weilerswist-Lommersum,Nordrhein-Westfalen
1572,1982-12-01,2020-02-19,80,50.9593,6.0392,Geilenkirchen (Flugplatz),Nordrhein-Westfalen
1590,1963-10-01,2020-02-19,37,51.4942,6.2463,Geldern-Walbeck,Nordrhein-Westfalen
1766,1982-01-01,2020-02-19,48,52.1344,7.6969,Münster/Osnabrück,Nordrhein-Westfalen


In [607]:
# Create variable with TRUE if state is Nordrhein-Westfalen
isNRW = df_stations_precip['state'] == "Nordrhein-Westfalen"

# Create variable with TRUE if date_to is latest date (indicates operation up to now)
isOperational = df_stations_precip['date_to'] == df_stations_precip.date_to.max() 

isBefore2010 = df_stations_precip['date_from'] < '2010'

# select on both conditions
dfNRW_precip = df_stations_precip[isNRW & isOperational & isBefore2010]
#print("Number of stations in NRW: \n", dfNRW.count())
dfNRW_precip

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
79,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
110,1931-01-01,2020-02-19,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen
187,1941-01-01,2020-02-19,265,51.4188,7.9126,Arnsberg-Holzen,Nordrhein-Westfalen
216,1971-01-01,2020-02-19,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
389,1931-01-01,2020-02-19,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen
...,...,...,...,...,...,...,...
13670,2007-06-01,2020-02-19,24,51.5088,6.7018,Duisburg-Baerl,Nordrhein-Westfalen
13671,2007-12-01,2020-02-19,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen
13696,2007-12-01,2020-02-19,60,51.5966,7.4049,Waltrop-Abdinghof,Nordrhein-Westfalen


In [608]:
# Xantern is in between Duisburg-Baerl and Kleve. These are the nearest stations
#dfNRW = dfNRW[(dfNRW['name']=="Duisburg-Baerl") | (dfNRW['name']=="Kleve")]

In [609]:
#These stations are within the radius of 40km from Xantern
#dfNRW = dfNRW[(dfNRW['name']=="Duisburg-Baerl") | (dfNRW['name']=="Geldern-Walbeck") | (dfNRW['name']=="Kleve")|(dfNRW['name']=="Borken in Westfalen")]

In [610]:
#These stations are within the radius of 50km of Xantern.
#dfNRW = dfNRW[(dfNRW['name']=="Duisburg-Baerl") | (dfNRW['name']=="Geldern-Walbeck") | (dfNRW['name']=="Kleve")|(dfNRW['name']=="Essen-Bredeney") | (dfNRW['name']=="Borken in Westfalen") | (dfNRW['name']=="Dusseldorf")|(dfNRW['name']=="Tönisvorst") | (dfNRW['name']=="Ahaus")]

In [611]:
 print(df_zips_temp)

                                                      name   ext    size type
station_id                                                                   
1           tageswerte_KL_00001_19370101_19860630_hist.zip  .zip  282024    -
3           tageswerte_KL_00003_18910101_20110331_hist.zip  .zip  887544    -
11          tageswerte_KL_00011_19800901_20181231_hist.zip  .zip   83682    -
44          tageswerte_KL_00044_19690101_20181231_hist.zip  .zip  416393    -
52          tageswerte_KL_00052_19690101_20011231_hist.zip  .zip  261747    -
...                                                    ...   ...     ...  ...
15963       tageswerte_KL_15963_19540101_20041130_hist.zip  .zip  337919    -
15965       tageswerte_KL_15965_19690501_19840831_hist.zip  .zip  101940    -
15979       tageswerte_KL_15979_19480101_19790331_hist.zip  .zip  221448    -
16085       tageswerte_KL_16085_19600701_19611231_hist.zip  .zip   16284    -
19087       tageswerte_KL_19087_19570501_19951130_hist.zip  .zip

In [612]:
 print(df_zips_precip)

                                                      name   ext    size type
station_id                                                                   
1           tageswerte_RR_00001_19120101_19860630_hist.zip  .zip  109677    -
2           tageswerte_RR_00002_19510101_20061231_hist.zip  .zip   82951    -
3           tageswerte_RR_00003_18910101_20110331_hist.zip  .zip  162410    -
4           tageswerte_RR_00004_19510101_19791031_hist.zip  .zip   45468    -
6           tageswerte_RR_00006_19821101_20181231_hist.zip  .zip   38084    -
...                                                    ...   ...     ...  ...
19042       tageswerte_RR_19042_18980601_19401231_hist.zip  .zip   63442    -
19043       tageswerte_RR_19043_19410101_19700731_hist.zip  .zip   44469    -
19044       tageswerte_RR_19044_19060901_20050331_hist.zip  .zip   87399    -
19086       tageswerte_RR_19086_18991001_19891231_hist.zip  .zip  131370    -
19087       tageswerte_RR_19087_19570501_19951130_hist.zip  .zip

In [613]:
 list(dfNRW_temp.index)

[390,
 617,
 1078,
 1300,
 1303,
 1327,
 1572,
 1590,
 1766,
 2110,
 2483,
 2497,
 2629,
 2667,
 2947,
 2968,
 3028,
 3031,
 3098,
 3321,
 3540,
 3591,
 3623,
 4063,
 4127,
 4371,
 5064,
 5347,
 5480,
 5717,
 6197,
 6264,
 6337,
 7106,
 7330,
 7374,
 7416,
 13670,
 13693,
 13696,
 13700,
 13713,
 13901,
 13952]

In [614]:
 list(dfNRW_precip.index)

[79,
 110,
 187,
 216,
 389,
 390,
 488,
 533,
 554,
 613,
 617,
 644,
 796,
 871,
 902,
 934,
 989,
 1024,
 1046,
 1078,
 1232,
 1241,
 1246,
 1277,
 1283,
 1298,
 1300,
 1303,
 1327,
 1590,
 1595,
 1673,
 1766,
 1891,
 1999,
 2027,
 2099,
 2104,
 2110,
 2117,
 2135,
 2254,
 2258,
 2332,
 2358,
 2419,
 2473,
 2483,
 2497,
 2505,
 2629,
 2667,
 2744,
 2802,
 2810,
 2936,
 2947,
 2968,
 2976,
 2999,
 3020,
 3028,
 3031,
 3081,
 3098,
 3202,
 3215,
 3264,
 3316,
 3321,
 3339,
 3350,
 3407,
 3465,
 3499,
 3540,
 3591,
 3610,
 3656,
 3767,
 3795,
 3798,
 3913,
 3952,
 4020,
 4063,
 4127,
 4150,
 4313,
 4368,
 4371,
 4400,
 4488,
 4667,
 4708,
 4741,
 4810,
 4849,
 4852,
 5064,
 5213,
 5271,
 5347,
 5360,
 5468,
 5480,
 5483,
 5486,
 5502,
 5513,
 5579,
 5594,
 5619,
 5699,
 5717,
 5733,
 6197,
 6264,
 6313,
 6337,
 7106,
 7330,
 7344,
 7374,
 7378,
 13669,
 13670,
 13671,
 13696,
 13700,
 13713]

In [615]:
# Add the names of the zip files only to a list. 
local_zip_list_temp = []

station_ids_selected_temp = list(dfNRW_temp.index)

for station_id in station_ids_selected_temp:
    try:
        fname = df_zips_temp["name"][station_id]
        print(fname)
        grabFile(ftp_dir_temp + fname, local_ftp_temp_ts_dir + fname)
        local_zip_list_temp.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

tageswerte_KL_00390_19861201_20181231_hist.zip
tageswerte_KL_00617_20040601_20181231_hist.zip
tageswerte_KL_01078_19520101_20181231_hist.zip
tageswerte_KL_01300_19380101_20181231_hist.zip
tageswerte_KL_01303_19350101_20181231_hist.zip
tageswerte_KL_01327_19370101_20181231_hist.zip
tageswerte_KL_01590_19631001_20181231_hist.zip
tageswerte_KL_01766_19820101_20181231_hist.zip
tageswerte_KL_02110_19630101_20181231_hist.zip
tageswerte_KL_02483_19261101_20181231_hist.zip
tageswerte_KL_02497_19470101_20181231_hist.zip
tageswerte_KL_02629_19471111_20181231_hist.zip
tageswerte_KL_02667_19570701_20181231_hist.zip
tageswerte_KL_02947_19620401_20181231_hist.zip
tageswerte_KL_02968_19450101_20181231_hist.zip
tageswerte_KL_03028_19510101_20181231_hist.zip
tageswerte_KL_03031_19801201_20181231_hist.zip
tageswerte_KL_03098_19931223_20181231_hist.zip
tageswerte_KL_03321_20080425_20181231_hist.zip
tageswerte_KL_03540_19920901_20181231_hist.zip
tageswerte_KL_03591_19940101_20181231_hist.zip
tageswerte_KL

In [616]:
# Add the names of the zip files only to a list. 
local_zip_list_precip = []

station_ids_selected_precip = list(dfNRW_precip.index)

for station_id in station_ids_selected_precip:
    try:
        fname = df_zips_precip["name"][station_id]
        print(fname)
        grabFile(ftp_dir_precip + fname, local_ftp_precip_ts_dir + fname)
        local_zip_list_precip.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

tageswerte_RR_00079_19310101_20181231_hist.zip
tageswerte_RR_00110_19310101_20181231_hist.zip
tageswerte_RR_00187_19410101_20181231_hist.zip
tageswerte_RR_00216_19710101_20181231_hist.zip
tageswerte_RR_00389_19310101_20181231_hist.zip
tageswerte_RR_00390_19861201_20181231_hist.zip
tageswerte_RR_00488_19410101_20181231_hist.zip
tageswerte_RR_00533_19610101_20181231_hist.zip
tageswerte_RR_00554_19460101_20181231_hist.zip
tageswerte_RR_00613_19410101_20181231_hist.zip
tageswerte_RR_00617_19410101_20181231_hist.zip
tageswerte_RR_00644_19410101_20181231_hist.zip
tageswerte_RR_00796_19410101_20181231_hist.zip
tageswerte_RR_00871_19410101_20181231_hist.zip
tageswerte_RR_00902_19310101_20181231_hist.zip
tageswerte_RR_00934_20041001_20181231_hist.zip
tageswerte_RR_00989_19310101_20181231_hist.zip
tageswerte_RR_01024_19310101_20181231_hist.zip
tageswerte_RR_01046_19710101_20181231_hist.zip
tageswerte_RR_01078_19690701_20181231_hist.zip
tageswerte_RR_01232_19410101_20181231_hist.zip
tageswerte_RR

In [617]:
def precip_ts_to_df(fname):
    
    dateparse = lambda dates: [pd.datetime.strptime(str(d), '%Y%m%d') for d in dates]

    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse, na_values = [-999.0, -999])

    #df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2",\
    #             index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse)
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
    return(df)

In [618]:
def temp_ts_to_df(fname):
    
    dateparse = lambda dates: [pd.datetime.strptime(str(d), '%Y%m%d') for d in dates]

    df = pd.read_csv(fname, delimiter=";", encoding="utf8", index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse, na_values = [-999.0, -999])

    #df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2",\
    #             index_col="MESS_DATUM", parse_dates = ["MESS_DATUM"], date_parser = dateparse)
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.index.name = df.index.name.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
    return(df)

In [619]:
from zipfile import ZipFile

In [620]:
def temp_ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list_temp:
        ffname = local_ftp_temp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip: 
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = temp_ts_to_df(myfile)
                s = dftmp["tmk"].rename(dftmp["stations_id"][0]).to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "time", inplace = True)   
    return(df)

In [621]:
def precip_ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list_precip:
        ffname = local_ftp_precip_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip: 
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dfprecip = precip_ts_to_df(myfile)
                s = dfprecip["rs"].rename(dfprecip["stations_id"][0]).to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "time", inplace = True)   
    return(df)

In [622]:
df_merged_ts_temp = temp_ts_merge()

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_00390_19861201_20181231_hist.zip
Extract product file: produkt_klima_tag_19861201_20181231_00390.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_00617_20040601_20181231_hist.zip
Extract product file: produkt_klima_tag_20040601_20181231_00617.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01078_19520101_20181231_hist.zip
Extract product file: produkt_klima_tag_19520101_20181231_01078.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01300_19380101_20181231_hist.zip
Extract product file: produkt_klima_tag_19380101_20181231_01300.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01303_19350101_20181231_hist.zip
Extract product file: produkt_klima_tag_19350101_20181231_01303.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01327_19370101_20181231_hist.zip
Extract product file: produkt_klima_tag_19370101_20181231_01327.txt

In [623]:
df_merged_ts_precip = precip_ts_merge()

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00079_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_00079.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00110_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_00110.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00187_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_00187.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00216_19710101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19710101_20181231_00216.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00389_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_00389.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00390_19861201_20181231_hist.zip
Extract 

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02483_19261101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19261101_20181231_02483.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02497_19470101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19470101_20181231_02497.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02505_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_02505.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02629_19471101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19471101_20181231_02629.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02667_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_02667.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02744_19410101_20181231_hist.zip
Extract 

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04708_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_04708.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04741_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_04741.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04810_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_04810.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04849_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_04849.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04852_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_04852.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_05064_20041201_20181231_hist.zip
Extract 

In [624]:
 df_merged_ts_temp.head()
    

,390,617,1078,1300,1303,1327,1590,1766,2110,2483,...,7330,7374,7416,13670,13693,13696,13700,13713,13901,13952
time,,,,,,,,,,,,,,,,,,,,,
1926-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [625]:
df_merged_ts_precip.head()

,79,110,187,216,389,390,488,533,554,613,...,7330,7344,7374,7378,13669,13670,13671,13696,13700,13713
time,,,,,,,,,,,,,,,,,,,,,
1891-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [626]:
df_merged_ts_precip

,79,110,187,216,389,390,488,533,554,613,...,7330,7344,7374,7378,13669,13670,13671,13696,13700,13713
time,,,,,,,,,,,,,,,,,,,,,
1891-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-28,0.2,0.2,1.2,0.1,0.2,0.0,1.4,0.0,0.1,0.4,...,0.8,0.0,0.1,0.0,0.3,0.2,0.2,0.2,0.3,0.5
2018-12-29,1.8,2.0,2.3,3.4,3.6,2.9,6.1,0.4,0.8,3.1,...,2.0,0.7,2.3,0.6,2.9,2.1,4.4,1.4,2.2,3.1


In [627]:
df_merged_ts_temp

,390,617,1078,1300,1303,1327,1590,1766,2110,2483,...,7330,7374,7416,13670,13693,13696,13700,13713,13901,13952
time,,,,,,,,,,,,,,,,,,,,,
1926-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,2.6,1.4,2.7,0.6,3.5,1.3,1.3,0.9,0.7,2.8,...,1.2,1.2,NaN,2.5,NaN,1.3,3.5,0.6,NaN,NaN
2018-12-28,1.0,0.3,1.5,-0.2,1.7,1.2,0.8,0.0,1.5,1.2,...,0.4,0.3,NaN,1.3,NaN,0.7,2.1,0.0,NaN,NaN
2018-12-29,1.6,5.6,5.6,3.1,4.7,5.6,5.5,5.7,5.2,0.7,...,6.0,5.9,NaN,5.5,NaN,5.8,4.7,2.3,NaN,NaN


In [628]:
# Create variable with TRUE if time is between 2015 and 2016
isdate2016_temp = df_merged_ts_temp.index < '2016-06-09'
isdate2016_precip = df_merged_ts_precip.index < '2016-06-09'

isdate2015_temp= df_merged_ts_temp.index > '2015-05-08'
isdate2015_precip= df_merged_ts_precip.index > '2015-05-08'

# select on both conditions
df_merged_ts_temp_2016 = df_merged_ts_temp[isdate2016_temp & isdate2015_temp]
df_merged_ts_precip_2016 = df_merged_ts_precip[isdate2016_precip & isdate2015_precip]
df_merged_ts_temp_2016
df_merged_ts_precip_2016

,79,110,187,216,389,390,488,533,554,613,...,7330,7344,7374,7378,13669,13670,13671,13696,13700,13713
time,,,,,,,,,,,,,,,,,,,,,
2015-05-09,0.0,0.8,0.4,1.7,0.7,1.3,1.3,1.8,0.1,0.2,...,0.4,0.3,0.4,0.9,0.9,1.2,2.5,0.5,0.0,2.1
2015-05-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-05-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-05-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-05-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-04,3.0,0.0,0.0,0.0,1.9,2.4,0.0,0.4,0.0,0.3,...,0.0,2.4,0.0,7.1,0.0,3.4,1.4,3.5,1.4,0.0
2016-06-05,1.5,0.0,0.0,0.0,0.1,0.1,0.0,2.3,0.0,0.5,...,0.0,2.6,0.0,0.0,0.0,0.0,15.1,0.0,0.0,0.0
2016-06-06,0.0,0.0,0.0,6.8,0.0,0.0,0.0,14.9,0.0,0.0,...,0.0,0.0,0.0,2.2,0.0,0.0,0.3,0.0,0.0,0.0


In [629]:
# Create variable with TRUE if time is between 2016 and 2017
isdate2017_temp = df_merged_ts_temp.index < '2017-05-27'
isdate2017_precip = df_merged_ts_precip.index < '2017-05-27'

isdate2016_temp= df_merged_ts_temp.index > '2016-05-26'
isdate2016_precip= df_merged_ts_precip.index > '2016-05-26'

# select on both conditions
df_merged_ts_temp_2017 = df_merged_ts_temp[isdate2017_temp & isdate2016_temp]
df_merged_ts_precip_2017 = df_merged_ts_precip[isdate2017_precip & isdate2016_precip]
df_merged_ts_temp_2017
df_merged_ts_precip_2017

,79,110,187,216,389,390,488,533,554,613,...,7330,7344,7374,7378,13669,13670,13671,13696,13700,13713
time,,,,,,,,,,,,,,,,,,,,,
2016-05-27,2.8,0.0,1.8,10.6,10.5,12.6,0.2,13.8,0.4,0.0,...,0.1,0.1,1.4,0.5,0.0,0.4,0.9,0.0,2.5,3.7
2016-05-28,0.5,0.3,0.8,2.1,17.1,11.0,0.0,7.8,0.5,6.0,...,0.0,0.0,0.3,1.7,1.5,0.5,0.0,0.0,0.0,0.0
2016-05-29,27.5,0.1,15.0,18.9,13.9,17.0,0.8,20.8,1.5,0.5,...,2.9,11.8,2.2,14.8,0.7,5.4,14.2,22.1,17.0,11.5
2016-05-30,8.8,1.0,1.2,5.4,18.6,21.9,3.0,9.9,21.0,3.1,...,1.2,26.8,4.0,24.0,2.3,20.1,15.3,19.0,25.9,4.5
2016-05-31,2.3,4.1,4.1,0.5,2.1,3.3,3.1,0.0,7.8,1.5,...,0.7,0.5,5.3,0.3,2.0,1.9,0.4,0.3,2.8,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-05-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-24,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [630]:
# Create variable with TRUE if time is between 2017 and 2018
isdate2018_temp = df_merged_ts_temp.index < '2018-05-09'
isdate2018_precip = df_merged_ts_precip.index < '2018-05-09'

isdate2017_temp= df_merged_ts_temp.index > '2017-05-08'
isdate2017_precip= df_merged_ts_precip.index > '2017-05-08'

# select on both conditions
df_merged_ts_temp_2018 = df_merged_ts_temp[isdate2018_temp & isdate2017_temp]
df_merged_ts_precip_2018 = df_merged_ts_precip[isdate2018_precip & isdate2017_precip]
df_merged_ts_temp_2018
df_merged_ts_precip_2018

,79,110,187,216,389,390,488,533,554,613,...,7330,7344,7374,7378,13669,13670,13671,13696,13700,13713
time,,,,,,,,,,,,,,,,,,,,,
2017-05-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-11,2.8,0.2,0.8,2.4,5.5,2.1,1.6,0.5,1.9,0.1,...,1.2,1.6,1.5,4.0,0.6,0.3,3.3,1.4,0.2,3.0
2017-05-12,1.1,2.2,3.6,1.0,4.2,8.7,4.1,1.1,16.1,3.1,...,8.7,3.9,2.8,2.3,5.5,17.5,0.2,0.7,6.2,6.2
2017-05-13,0.9,0.0,0.5,0.6,0.0,2.6,0.3,0.0,0.0,5.5,...,0.3,1.2,0.0,0.0,0.1,0.1,1.6,0.0,0.5,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [631]:
df_merged_ts_precip_2018

,79,110,187,216,389,390,488,533,554,613,...,7330,7344,7374,7378,13669,13670,13671,13696,13700,13713
time,,,,,,,,,,,,,,,,,,,,,
2017-05-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-11,2.8,0.2,0.8,2.4,5.5,2.1,1.6,0.5,1.9,0.1,...,1.2,1.6,1.5,4.0,0.6,0.3,3.3,1.4,0.2,3.0
2017-05-12,1.1,2.2,3.6,1.0,4.2,8.7,4.1,1.1,16.1,3.1,...,8.7,3.9,2.8,2.3,5.5,17.5,0.2,0.7,6.2,6.2
2017-05-13,0.9,0.0,0.5,0.6,0.0,2.6,0.3,0.0,0.0,5.5,...,0.3,1.2,0.0,0.0,0.1,0.1,1.6,0.0,0.5,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [632]:
df_merged_ts_temp_2018.to_csv(local_ts_merged_dir + "df_new_merged_ts_temp_2018.csv",sep=";")
df_merged_ts_temp_2016.to_csv(local_ts_merged_dir + "df_new_merged_ts_temp_2016.csv",sep=";")
df_merged_ts_temp_2017.to_csv(local_ts_merged_dir + "df_new_merged_ts_temp_2017.csv",sep=";")
df_merged_ts_precip_2018.to_csv(local_ts_merged_dir + "df_new_merged_ts_precip_2018.csv",sep=";")
df_merged_ts_precip_2016.to_csv(local_ts_merged_dir + "df_new_merged_ts_precip_2016.csv",sep=";")
df_merged_ts_precip_2017.to_csv(local_ts_merged_dir + "df_new_merged_ts_precip_2017.csv",sep=";")

In [633]:
df_merged_ts_transposed_temp_2016 = df_merged_ts_temp_2016.transpose()
df_merged_ts_transposed_temp_2017 = df_merged_ts_temp_2017.transpose()
df_merged_ts_transposed_temp_2018 = df_merged_ts_temp_2018.transpose()
df_merged_ts_transposed_precip_2016 = df_merged_ts_precip_2016.transpose()
df_merged_ts_transposed_precip_2017 = df_merged_ts_precip_2017.transpose()
df_merged_ts_transposed_precip_2018 = df_merged_ts_precip_2018.transpose()

In [634]:
df_merged_ts_transposed.index.names = ['station_id']

NameError: name 'df_merged_ts_transposed' is not defined

In [635]:
df_merged_ts_transposed.head()

NameError: name 'df_merged_ts_transposed' is not defined

In [636]:
df_merged_ts_transposed_temp_2016.to_csv(local_ts_merged_dir + "ts_merged_transposed_temp_2016.csv",sep=";")
df_merged_ts_transposed_temp_2017.to_csv(local_ts_merged_dir + "ts_merged_transposed_temp_2017.csv",sep=";")
df_merged_ts_transposed_temp_2018.to_csv(local_ts_merged_dir + "ts_merged_transposed_temp_2018.csv",sep=";")
df_merged_ts_transposed_precip_2016.to_csv(local_ts_merged_dir + "ts_merged_transposed_precip_2016.csv",sep=";")
df_merged_ts_transposed_precip_2017.to_csv(local_ts_merged_dir + "ts_merged_transposed_precip_2017.csv",sep=";")
df_merged_ts_transposed_precip_2018.to_csv(local_ts_merged_dir + "ts_merged_transposed_precip_2018.csv",sep=";")

In [637]:
 def ts_append_temp():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list_temp:
        ffname = local_ftp_temp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = temp_ts_to_df(myfile)
                dftmp = dftmp.merge(df_stations,how="inner",left_on="stations_id",right_on="station_id",right_index=True)
#                print(dftmp.head(5))
                df = df.append(dftmp)

    #df.index.names = ["year"]
    #df.index.rename(name = "time", inplace = True)
    return(df)

In [638]:
def ts_append_precip():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list_precip:
        ffname = local_ftp_precip_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dfprecip = precip_ts_to_df(myfile)
                dfprecip = dfprecip.merge(df_stations,how="inner",left_on="stations_id",right_on="station_id",right_index=True)
#                print(dftmp.head(5))
                df = df.append(dfprecip)

    #df.index.names = ["year"]
    #df.index.rename(name = "time", inplace = True)
    return(df)

In [639]:
df_appended_ts_temp = ts_append_temp()


Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_00390_19861201_20181231_hist.zip
Extract product file: produkt_klima_tag_19861201_20181231_00390.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_00617_20040601_20181231_hist.zip
Extract product file: produkt_klima_tag_20040601_20181231_00617.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01078_19520101_20181231_hist.zip
Extract product file: produkt_klima_tag_19520101_20181231_01078.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01300_19380101_20181231_hist.zip
Extract product file: produkt_klima_tag_19380101_20181231_01300.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01303_19350101_20181231_hist.zip
Extract product file: produkt_klima_tag_19350101_20181231_01303.txt

Zip archive: data/original/DWD/daily/kl/historical/tageswerte_KL_01327_19370101_20181231_hist.zip
Extract product file: produkt_klima_tag_19370101_20181231_01327.txt

In [640]:
df_appended_ts_precip = ts_append_precip()

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00079_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_00079.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00110_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_00110.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00187_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_00187.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00216_19710101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19710101_20181231_00216.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00389_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_00389.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_00390_19861201_20181231_hist.zip
Extract 

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02483_19261101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19261101_20181231_02483.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02497_19470101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19470101_20181231_02497.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02505_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_02505.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02629_19471101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19471101_20181231_02629.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02667_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_02667.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_02744_19410101_20181231_hist.zip
Extract 

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04708_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_04708.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04741_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_04741.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04810_19310101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19310101_20181231_04810.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04849_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_04849.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_04852_19410101_20181231_hist.zip
Extract product file: produkt_nieder_tag_19410101_20181231_04852.txt

Zip archive: data/original/DWD/daily/more_precip/historical/tageswerte_RR_05064_20041201_20181231_hist.zip
Extract 

In [641]:
df_appended_ts_temp

,stations_id,qn_3,fx,fm,qn_4,rsk,rskf,sdk,shk_tag,nm,...,tnk,tgk,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,,,,,,,,,
1986-12-01,390,NaN,NaN,NaN,10.0,1.2,6.0,6.0,0.0,5.7,...,-5.0,-7.0,eor,1986-12-01,2020-02-19,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
1986-12-02,390,NaN,NaN,NaN,10.0,0.8,6.0,0.0,0.0,8.0,...,-1.6,-4.5,eor,1986-12-01,2020-02-19,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
1986-12-03,390,NaN,NaN,NaN,10.0,0.0,0.0,3.0,0.0,3.0,...,1.5,2.0,eor,1986-12-01,2020-02-19,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
1986-12-04,390,NaN,NaN,NaN,10.0,0.0,0.0,1.3,0.0,4.3,...,2.1,-2.0,eor,1986-12-01,2020-02-19,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
1986-12-05,390,NaN,NaN,NaN,10.0,1.2,6.0,7.3,0.0,1.7,...,1.5,0.0,eor,1986-12-01,2020-02-19,610,50.9837,8.3683,"Berleburg, Bad-Stünzel",Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,13713,NaN,NaN,NaN,3.0,0.0,0.0,NaN,0.0,NaN,...,-2.8,-5.2,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-12-28,13713,NaN,NaN,NaN,3.0,0.5,4.0,NaN,0.0,NaN,...,-3.7,-7.1,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-12-29,13713,NaN,NaN,NaN,3.0,3.1,4.0,NaN,0.0,NaN,...,0.2,0.2,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen


In [642]:
df_appended_ts_precip

,stations_id,qn_6,rs,rsf,sh_tag,nsh_tag,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,,
1931-01-01,79,5.0,5.8,8.0,NaN,NaN,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
1931-01-02,79,5.0,3.3,1.0,NaN,NaN,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
1931-01-03,79,5.0,5.4,8.0,NaN,NaN,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
1931-01-04,79,5.0,0.2,1.0,NaN,NaN,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
1931-01-05,79,5.0,2.0,8.0,NaN,NaN,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,13713,3.0,0.0,0.0,0.0,0.0,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-12-28,13713,3.0,0.5,4.0,0.0,0.0,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-12-29,13713,3.0,3.1,4.0,0.0,0.0,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen


In [643]:
# Create variable with TRUE if time is between 2015 and 2016
isdate2016_temp = df_appended_ts_temp.index < '2016-06-09'
isdate2016_precip = df_appended_ts_precip.index < '2016-06-09'

isdate2015_temp= df_appended_ts_temp.index > '2015-05-08'
isdate2015_precip= df_appended_ts_precip.index > '2015-05-08'

# select on both conditions
df_appended_ts_temp_2016 = df_appended_ts_temp[isdate2016_temp & isdate2015_temp]
df_appended_ts_temp_2016
df_appended_ts_precip_2016 = df_appended_ts_precip[isdate2016_precip & isdate2015_precip]
df_appended_ts_precip_2016

,stations_id,qn_6,rs,rsf,sh_tag,nsh_tag,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,,
2015-05-09,79,9.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2015-05-10,79,9.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2015-05-11,79,9.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2015-05-12,79,9.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2015-05-13,79,9.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-04,13713,9.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2016-06-05,13713,9.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2016-06-06,13713,9.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen


In [644]:
isdate2017_temp = df_appended_ts_temp.index < '2017-05-27'
isdate2017_precip = df_appended_ts_precip.index < '2017-05-27'

isdate2016_temp= df_appended_ts_temp.index > '2016-05-26'
isdate2016_precip= df_appended_ts_precip.index > '2016-05-26'

# select on both conditions
df_appended_ts_temp_2017 = df_appended_ts_temp[isdate2017_temp & isdate2016_temp]
df_appended_ts_temp_2017
df_appended_ts_precip_2017 = df_appended_ts_precip[isdate2017_precip & isdate2016_precip]
df_appended_ts_precip_2017

,stations_id,qn_6,rs,rsf,sh_tag,nsh_tag,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,,
2016-05-27,79,9.0,2.8,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2016-05-28,79,9.0,0.5,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2016-05-29,79,9.0,27.5,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2016-05-30,79,9.0,8.8,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2016-05-31,79,9.0,2.3,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-05-22,13713,3.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2017-05-23,13713,3.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2017-05-24,13713,3.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen


In [645]:
isdate2018_temp = df_appended_ts_temp.index < '2018-05-09'
isdate2018_precip = df_appended_ts_precip.index < '2018-05-09'

isdate2017_temp= df_appended_ts_temp.index > '2017-05-08'
isdate2017_precip= df_appended_ts_precip.index > '2017-05-08'

# select on both conditions
df_appended_ts_temp_2018 = df_appended_ts_temp[isdate2018_temp & isdate2017_temp]
df_appended_ts_temp_2018
df_appended_ts_precip_2018 = df_appended_ts_precip[isdate2018_precip & isdate2017_precip]
df_appended_ts_precip_2018

,stations_id,qn_6,rs,rsf,sh_tag,nsh_tag,eor,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,,,,,,
2017-05-09,79,3.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-10,79,3.0,0.0,0.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-11,79,3.0,2.8,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-12,79,3.0,1.1,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-13,79,3.0,0.9,6.0,0.0,0.0,eor,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-04,13713,3.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-05-05,13713,3.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-05-06,13713,3.0,0.0,0.0,NaN,NaN,eor,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen


In [646]:
df_appended_ts_temp_2016 = df_appended_ts_temp_2016.drop(['qn_3','fx','fm','qn_4','rsk','rskf','sdk','shk_tag','nm','vpm','pm','upm','txk','tnk','tgk','eor'], axis=1)
df_appended_ts_temp_2017 = df_appended_ts_temp_2017.drop(['qn_3','fx','fm','qn_4','rsk','rskf','sdk','shk_tag','nm','vpm','pm','upm','txk','tnk','tgk','eor'], axis=1)
df_appended_ts_temp_2018 = df_appended_ts_temp_2018.drop(['qn_3','fx','fm','qn_4','rsk','rskf','sdk','shk_tag','nm','vpm','pm','upm','txk','tnk','tgk','eor'], axis=1)
df_appended_ts_precip_2016 = df_appended_ts_precip_2016.drop(['qn_6','rsf','sh_tag','nsh_tag','eor'], axis=1)
df_appended_ts_precip_2017 = df_appended_ts_precip_2017.drop(['qn_6','rsf','sh_tag','nsh_tag','eor'], axis=1)
df_appended_ts_precip_2018 = df_appended_ts_precip_2018.drop(['qn_6','rsf','sh_tag','nsh_tag','eor'], axis=1)

In [647]:
df_appended_ts_temp_2016.to_csv(local_ts_appended_dir + "ts_appended_temp_2016.csv",sep=";")
df_appended_ts_temp_2017.to_csv(local_ts_appended_dir + "ts_appended_temp_2017.csv",sep=";")
df_appended_ts_temp_2018.to_csv(local_ts_appended_dir + "ts_appended_temp_2018.csv",sep=";")
df_appended_ts_precip_2016.to_csv(local_ts_appended_dir + "ts_appended_precip_2016.csv",sep=";")
df_appended_ts_precip_2017.to_csv(local_ts_appended_dir + "ts_appended_precip_2017.csv",sep=";")
df_appended_ts_precip_2018.to_csv(local_ts_appended_dir + "ts_appended_precip_2018.csv",sep=";")

In [648]:
df_appended_ts_temp_2016_mean = df_appended_ts_temp_2016.groupby('name').mean()
df_appended_ts_temp_2016_mean = df_appended_ts_temp_2016_mean[df_appended_ts_temp_2016_mean['tmk'].notna()]
df_appended_ts_temp_2017_mean = df_appended_ts_temp_2017.groupby('name').mean()
df_appended_ts_temp_2017_mean = df_appended_ts_temp_2017_mean[df_appended_ts_temp_2017_mean['tmk'].notna()]
df_appended_ts_temp_2018_mean = df_appended_ts_temp_2018.groupby('name').mean()
df_appended_ts_temp_2018_mean = df_appended_ts_temp_2018_mean[df_appended_ts_temp_2018_mean['tmk'].notna()]

In [669]:
df_appended_ts_precip_2016 = df_appended_ts_precip_2016[df_appended_ts_precip_2016['rs'].notna()]
df_appended_ts_precip_2017 = df_appended_ts_precip_2017[df_appended_ts_precip_2017['rs'].notna()]
df_appended_ts_precip_2018 = df_appended_ts_precip_2018[df_appended_ts_precip_2018['rs'].notna()]

In [670]:
df_appended_ts_precip_2018 

,stations_id,rs,date_from,date_to,altitude,latitude,longitude,name,state
mess_datum,,,,,,,,,
2017-05-09,79,0.0,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-10,79,0.0,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-11,79,2.8,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-12,79,1.1,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
2017-05-13,79,0.9,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen
...,...,...,...,...,...,...,...,...,...
2018-05-04,13713,0.0,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-05-05,13713,0.0,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
2018-05-06,13713,0.0,2007-11-01,2020-02-19,380,51.0899,7.6290,Meinerzhagen-Redlendorf,Nordrhein-Westfalen


In [672]:
#df_appended_ts_precip_2016['cumulative_sum'] = df_appended_ts_precip_2016.groupby(['name'])['rs'].cumsum().round(decimals = 3)
#df_appended_ts_precip_2016_cumulativesum = df_appended_ts_precip_2016
#df_appended_ts_precip_2017['cumulative_sum'] = df_appended_ts_precip_2017.groupby(['name'])['rs'].cumsum().round(decimals = 3)
#df_appended_ts_precip_2017_cumulativesum = df_appended_ts_precip_2017
#df_appended_ts_precip_2018['cumulative_sum'] = df_appended_ts_precip_2018.groupby(['name'])['rs'].cumsum().round(decimals = 3)
#df_appended_ts_precip_2018_cumulativesum = df_appended_ts_precip_2018
df_appended_ts_precip_2016_cumulative_sum = df_appended_ts_precip_2016.groupby(['stations_id','date_from','date_to','altitude','latitude','longitude','name','state']).sum().groupby('name').cumsum().round(decimals = 3)
df_appended_ts_precip_2017_cumulative_sum = df_appended_ts_precip_2017.groupby(['stations_id','date_from','date_to','altitude','latitude','longitude','name','state']).sum().groupby('name').cumsum().round(decimals = 3)
df_appended_ts_precip_2018_cumulative_sum = df_appended_ts_precip_2018.groupby(['stations_id','date_from','date_to','altitude','latitude','longitude','name','state']).sum().groupby('name').cumsum().round(decimals = 3)

In [673]:
df_appended_ts_precip_2016_cumulative_sum

,,,,,,,,rs
stations_id,date_from,date_to,altitude,latitude,longitude,name,state,
79,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen,864.5
110,1931-01-01,2020-02-19,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen,978.1
187,1941-01-01,2020-02-19,265,51.4188,7.9126,Arnsberg-Holzen,Nordrhein-Westfalen,1103.8
216,1971-01-01,2020-02-19,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen,1229.6
389,1931-01-01,2020-02-19,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen,1062.9
...,...,...,...,...,...,...,...,...
13670,2007-06-01,2020-02-19,24,51.5088,6.7018,Duisburg-Baerl,Nordrhein-Westfalen,868.4
13671,2007-12-01,2020-02-19,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen,1168.4
13696,2007-12-01,2020-02-19,60,51.5966,7.4049,Waltrop-Abdinghof,Nordrhein-Westfalen,967.4


In [674]:
df_appended_ts_temp_2016_mean.head(40)

,stations_id,tmk,altitude,latitude,longitude
name,,,,,
Ahaus,7374,11.104,46,52.0814,6.9410
Arnsberg-Neheim,7330,10.481,159,51.4633,7.9780
"Berleburg, Bad-Stünzel",390,8.194,610,50.9837,8.3683
Bielefeld-Deppendorf,7106,10.697,105,52.0714,8.4565
Borken in Westfalen,617,11.179,47,51.8730,6.8863
Brilon-Thülen,6264,9.328,457,51.4140,8.6500
Duisburg-Baerl,13670,12.057,24,51.5088,6.7018
Düsseldorf,1078,11.857,37,51.2960,6.7686
Eslohe,1300,6.242,351,51.2540,8.1565


In [675]:
df_appended_ts_precip_2016_cumulative_sum

,,,,,,,,rs
stations_id,date_from,date_to,altitude,latitude,longitude,name,state,
79,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen,864.5
110,1931-01-01,2020-02-19,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen,978.1
187,1941-01-01,2020-02-19,265,51.4188,7.9126,Arnsberg-Holzen,Nordrhein-Westfalen,1103.8
216,1971-01-01,2020-02-19,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen,1229.6
389,1931-01-01,2020-02-19,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen,1062.9
...,...,...,...,...,...,...,...,...
13670,2007-06-01,2020-02-19,24,51.5088,6.7018,Duisburg-Baerl,Nordrhein-Westfalen,868.4
13671,2007-12-01,2020-02-19,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen,1168.4
13696,2007-12-01,2020-02-19,60,51.5966,7.4049,Waltrop-Abdinghof,Nordrhein-Westfalen,967.4


In [676]:
df_appended_ts_temp_2016_mean['tmk'] = df_appended_ts_temp_2016_mean['tmk'].round(decimals = 3)
df_appended_ts_temp_2017_mean['tmk'] = df_appended_ts_temp_2017_mean['tmk'].round(decimals = 3)
df_appended_ts_temp_2018_mean['tmk'] = df_appended_ts_temp_2018_mean['tmk'].round(decimals = 3)

In [677]:
df_appended_ts_temp_2016_mean.to_csv(local_ts_appended_dir + "ts_appended_temp_2016_mean.csv",sep=";")
df_appended_ts_temp_2017_mean.to_csv(local_ts_appended_dir + "ts_appended_temp_2017_mean.csv",sep=";")
df_appended_ts_temp_2018_mean.to_csv(local_ts_appended_dir + "ts_appended_temp_2018_mean.csv",sep=";")
df_appended_ts_precip_2016_cumulative_sum.to_csv(local_ts_appended_dir + "ts_appended_precip_2016_cumulativesum.csv",sep=";")
df_appended_ts_precip_2017_cumulative_sum.to_csv(local_ts_appended_dir + "ts_appended_precip_2017_cumulativesum.csv",sep=";")
df_appended_ts_precip_2018_cumulative_sum.to_csv(local_ts_appended_dir + "ts_appended_precip_2018_cumulativesum.csv",sep=";")

In [678]:
df_appended_ts_precip_2016_cumulative_sum

,,,,,,,,rs
stations_id,date_from,date_to,altitude,latitude,longitude,name,state,
79,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen,864.5
110,1931-01-01,2020-02-19,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen,978.1
187,1941-01-01,2020-02-19,265,51.4188,7.9126,Arnsberg-Holzen,Nordrhein-Westfalen,1103.8
216,1971-01-01,2020-02-19,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen,1229.6
389,1931-01-01,2020-02-19,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen,1062.9
...,...,...,...,...,...,...,...,...
13670,2007-06-01,2020-02-19,24,51.5088,6.7018,Duisburg-Baerl,Nordrhein-Westfalen,868.4
13671,2007-12-01,2020-02-19,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen,1168.4
13696,2007-12-01,2020-02-19,60,51.5966,7.4049,Waltrop-Abdinghof,Nordrhein-Westfalen,967.4


In [679]:
df_appended_ts_precip_2017_cumulative_sum

,,,,,,,,rs
stations_id,date_from,date_to,altitude,latitude,longitude,name,state,
79,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen,614.5
110,1931-01-01,2020-02-19,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen,691.1
187,1941-01-01,2020-02-19,265,51.4188,7.9126,Arnsberg-Holzen,Nordrhein-Westfalen,691.2
216,1971-01-01,2020-02-19,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen,808.0
389,1931-01-01,2020-02-19,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen,709.9
...,...,...,...,...,...,...,...,...
13670,2007-06-01,2020-02-19,24,51.5088,6.7018,Duisburg-Baerl,Nordrhein-Westfalen,673.7
13671,2007-12-01,2020-02-19,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen,951.0
13696,2007-12-01,2020-02-19,60,51.5966,7.4049,Waltrop-Abdinghof,Nordrhein-Westfalen,627.3


In [680]:
df_appended_ts_precip_2018_cumulative_sum

,,,,,,,,rs
stations_id,date_from,date_to,altitude,latitude,longitude,name,state,
79,1931-01-01,2020-02-19,160,50.6718,7.0155,Alfter-Volmershoven,Nordrhein-Westfalen,662.4
110,1931-01-01,2020-02-19,65,52.0487,7.4877,Altenberge,Nordrhein-Westfalen,817.3
187,1941-01-01,2020-02-19,265,51.4188,7.9126,Arnsberg-Holzen,Nordrhein-Westfalen,1037.1
216,1971-01-01,2020-02-19,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen,1088.7
389,1931-01-01,2020-02-19,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen,972.1
...,...,...,...,...,...,...,...,...
13670,2007-06-01,2020-02-19,24,51.5088,6.7018,Duisburg-Baerl,Nordrhein-Westfalen,737.2
13671,2007-12-01,2020-02-19,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen,1150.5
13696,2007-12-01,2020-02-19,60,51.5966,7.4049,Waltrop-Abdinghof,Nordrhein-Westfalen,828.9
